# Projeto 1 - Ciência dos Dados

### Objetivo do Projeto

Você foi contratado por uma empresa para analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você: crie um programa que selecione algumas mensagens disponíveis no Twitter, as quais mencionam esse particular produto; e classifique esses tweets como "relevante" ou "irrelevante", pelo menos.Com isso, essa empresa deseja que mensagens relevantes, que denigrem o nome do produto, ou que mereçam destaque, por exemplo, disparem um foco de atenção da área de marketing. Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails, por exemplo. Esse classificador permite calcular qual a probabilidade de uma mensagem ser relevante dada as palavras em seu conteúdo.

Para realizar o MVP (minimum viable product) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.

Após validado, o seu protótipo poderia, porque não, também capturar e classificar automaticamente as mensagens da plataforma.


#### Contribuidores

Nome: Larissa Jordana de Paula Soares

Nome: Luis Antônio Bordignon de Oliveira

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
import unidecode
import unicodedata
import re

### Diretório utilizado:

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\LariLari\Projeto1_Cdados


### Carregando a base de dados com os tweets classificados como (R) relevantes e (I) não relevantes do excel de treinamento e teste:

In [3]:
filename = 'sony.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(6)

,Treinamento,Classificação
0,se o martinho da vila pedir 01 dólar por cada ...,I
1,@jeanino_ @s9_ricardo @fuck_off_matt então a s...,I
2,2022 vai me falir dmsssss pqp caralho https://...,I
3,"@luarzinmsm ata, a sony é uma vagabunda mesmo",I
4,@rafael_fenixx @raphaelvieirac4 @drakesincero1...,I
5,@formigonny acho que vai ser só sony viu...,I


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Clasificação
0,único jogo novo anunciado pela sony que eu que...,R
1,@highlevelbeu sony no seu nivel mais belo né k...,I
2,a sony vai lançar uma sequência de jogos mt fo...,I
3,@th0rdthy gloria a sony,I
4,"@paladinodoxbox sony liberou a versão do ps5, ...",I


___
## Classificador automático de sentimento


 ##### Critérios de classificação dos tweets:
  Foram consideradas relevantes críticas à empresa sony (com contexto detectável), como a insatisfação do público por causa do pequeno tempo de duração de um filme produzido pela empresa. Também, em menor quantidade, elogios certeiros foram levados em consideração, apontando aonde a empresa acertou e deve continuar assim.


___
## Montando um Classificador Naive-Bayes

##### Contextualização:

  O algoritmo “Naive Bayes” é um classificador probabilístico muito utilizado em machine learning. Baseado no “Teorema de Bayes”, o modelo foi criado por um matemático inglês, e também ministro presibiteriano, chamado Thomas Bayes (1701 – 1761) para tentar provar a existência de Deus. Hoje é também utilizado na área de Aprendizado de Máquina (Machine Learning) para categorizar textos com base na frequência das palavras usadas. Entre as possibilidades de aplicações está a classificação de um e-mail como SPAM ou Não-SPAM e a identificação de um assunto com base em seu conteúdo. Ele recebe o nome de “naive” (ingênuo) porque desconsidera a correlação entre as variáveis (features). Ou seja, se determinada fruta é rotulada como “Limão”, caso ela também seja descrita como “Verde” e “Redonda”, o algoritmo não vai levar em consideração a correlação entre esses fatores. Isso porque trata cada um de forma independente.
  
  
  
  Fontes: https://www.datageeks.com.br/naive-bayes/
  
  
  O modelo Naive Bayes é fácil de construir e particularmente útil para grandes volumes de dados. Além de simples, Naive Bayes é conhecido por ganhar de métodos de classificação altamente sofisticados. Teorema de Bayes fornece uma forma de calcular a probabilidade posterior P (C | X) a partir de P (C), P (x) e P (X | c). Note a equação abaixo:
  
  $P(c|x) = \frac{P(x|c) \cdot P(c)}{P(x)}$ $\quad$
  
  
  $\quad\quad P(c|X) = P(x1|c) \cdot P(x2|c) \cdot P(x3|c) \cdot P(x4|C) \cdot P(x5|c) \cdot ... \cdot P(xN|c) \cdot P(c)$
  
  
 * P (c | x) é a probabilidade posterior da classe (c, alvo) dada preditor (x, atributos).
 * P (c) é a probabilidade original da classe.
 * P (x | c) é a probabilidade que representa a probabilidade de preditor dada a classe.
 * P (x) é a probabilidade original do preditor.
 
 
 
  Fontes: https://www.vooo.pro/insights/6-passos-faceis-para-aprender-o-algoritmo-naive-bayes-com-o-codigo-em-python/
  


#### Agora para o nosso projeto:
Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
# Arquivo "sony"
train.head()

,Treinamento,Classificação
0,se o martinho da vila pedir 01 dólar por cada ...,I
1,@jeanino_ @s9_ricardo @fuck_off_matt então a s...,I
2,2022 vai me falir dmsssss pqp caralho https://...,I
3,"@luarzinmsm ata, a sony é uma vagabunda mesmo",I
4,@rafael_fenixx @raphaelvieirac4 @drakesincero1...,I


## Implementação a devida Limpeza no Twitters:

In [7]:
#célula de limpeza
import functools
import operator
def cleanup(text):#Função de limpeza muito simples que troca alguns sinais básicos por espaços
    #import string
    punctuation = '[!-.:?;"”“*)(/|[\]]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = emoji.get_emoji_regexp().split(text_subbed)
    separa = [texto.split() for texto in text_subbed]
    text_subbed = functools.reduce(operator.concat, separa)
    return " ".join(text_subbed)

#Função que coloca todos texto em minúsculo 
def lower(text):
    return text.lower()

#Função que remove os @ e o "http"
def remove_imagem_arroba(text): 
    lista = ""
    for i in text.split():
        if i[:4]!= 'http' and i[0]!='@':
            lista+= i + " "
    return lista.strip()

# Função que remove acentuação
def remove_acentos(text):
    text = unicodedata.normalize('NFD', text)
    return re.sub(r'[\u0300-\u036f]', '', text).casefold()


In [8]:
#Função com a implementação de todas a limpezas
def limpeza (text):
    text = remove_acentos(text)
    text= lower(text)
    text = remove_imagem_arroba(text)
    text = cleanup(text)
    return text

In [9]:
num= 0
for i in train.loc[:,'Treinamento']:
    train.loc[num, 'Treinamento']= limpeza(i)
    num+= 1
train

,Treinamento,Classificação
0,se o martinho da vila pedir 01 dolar por cada ...,I
1,entao a sony n revela os numeros de tlou pq fr...,I
2,2022 vai me falir dmsssss pqp caralho,I
3,ata a sony e uma vagabunda mesmo,I
4,ao decorrer das decadas sony god of war 1° fra...,I
...,...,...
295,q na verdade ja tem ne pq em 2017 passava na s...,I
296,relevancia me diga qual foi o ultimo jogo inov...,R
297,thor gordo isso mesmo sony sempre fiel a mitol...,I
298,estranho e saber que a r conseguiu ser a unica...,I


### Primeira parte do desenvolvimento:
 Para essa segunda interação, vamos introduzir alguns calculos importantes. Sendo eles:
 * Calculo do total de palavras separadas dos tweets relevantes e irrelevantes.
 * Calculo do total de palavras separadas de dos os tweets.
 

In [10]:
#Separando as mensagens Relevantes e Irrelevantes da nossa base de treinamento
Train_Relevante= train.loc[train['Classificação']=='R', 'Treinamento']
Train_Irrelevante= train.loc[train['Classificação']=='I', 'Treinamento']
Train_Total= train.loc[:,'Treinamento']
train.shape[0]

300

In [11]:
# Implementando a limpeza
l_relevante=[]
l_irrelevante=[]
l_total=[]

for i in Train_Relevante:
    l_relevante.append(i)
l_relevante= " ".join(l_relevante).split()

for i in Train_Irrelevante:
     l_irrelevante.append(i)
l_irrelevante= " ".join(l_irrelevante).split()
for i in Train_Total:
     l_total.append(i)
l_total= " ".join(l_total).split()

In [12]:
tab_relevante  = pd.Series(l_relevante)
tab_irrelevante  = pd.Series(l_irrelevante)
tab_total  = pd.Series(l_total)

###  Fazendo cálculo de frequências:

In [13]:
#Frequencias absolutas
tab_rel_absoluta  = tab_relevante.value_counts()
tab_irr_absoluta  = tab_irrelevante.value_counts()
tab_total_absoluta = tab_total.value_counts()

In [14]:
# Frequências relativas
tab_rel_relativa = tab_relevante.value_counts(True)
tab_irr_relativa = tab_irrelevante.value_counts(True)
tab_total_relativa = tab_total.value_counts(True)

In [15]:
palavras_separadas_total=[]
for i in l_total:
    if i not in palavras_separadas_total:
        palavras_separadas_total.append(i)

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Segundo desenvolvimento:

##### Calcular a $P(R)$ e $P(I)$:
   Sendo ele: 
   * $\quad\quad P(R) =$ $\frac{R}{T}$ (R= total de palavras dos tweets relevantes e T= total de palavras de todos os tweets)
   * $\quad\quad P(R) =$ $\frac{I}{T}$ (I= total de palavras dos tweets irrelevantes e T= total de palavras de todos os tweets)
   
   
##### em seguida:
  Calcular $P(T|R)$ e $P(T|I)$
  
  Sendo:
  * $\quad\quad\quad P(palavra|R) = \frac{Tr}{R}$
  $\quad\quad\quad Tr=$ total de vezes que a palavra apareceu nos tweets relevantes
  
  
  $\quad\quad P(T|R) = P(palavra1|R) \cdot P(palavra2|R) \cdot P(palavra3|R) \cdot P(palavra4|R) \cdot P(palavra5|R) \cdot ... \cdot P(palavraN|R)$





In [16]:
#Calculo de P(R) e P(I)
probR = sum(tab_rel_absoluta) / sum(tab_total_absoluta)

probI = sum(tab_irr_absoluta) / sum(tab_total_absoluta)

In [17]:
#Fazendo a limpeza
l_test= []
for i in test.Teste:
    l_test.append(limpeza(i).split())

In [18]:
# criando um DataFrame para o computador preencher

df = {'TweetN':  [],
        'N_Classificação': [],
        }
NovoDF = pd.DataFrame(df, columns = ['TweetN','N_Classificação'])

### Terceiro Desenvolvimento:

#####  Calcular as probabilidades com a suavização de laplace tanto para os relevantes, quanto para os irrelevantes, como mostrado abaixo:


 Para relevantes:
 * $\quad P(R|T) = \frac{P(T \cap R)}{P(T)}$ $\Rightarrow$ $P(R|T) = \frac{P(T|R) \cdot P(R)}{P(T)}$ $\quad$
 
 
 Para irrelevantes:
 * $\quad P(I|T) = \frac{P(T \cap I)}{P(T)}$ $\Rightarrow$ $P(I|T) = \frac{P(T|I) \cdot P(I)}{P(T)}$ $\quad$
 
 
 ###### Assim, descobrindo e comparando as probabilidades pela definição dos tweet (R) para relevante e (I) para irrelevante:
 
 
 * $P(R|T)$ 

 * $P(I|T)$
 
 * $\quad  P(R|T) > P(I|T)$ ou $P(R|T) < P(I|T)$ $\quad$

In [19]:
# #Calculando a probabilidade de uma frase ser R ou I por Naive Bayes e a suavização de laplace

# contador que acessa a linha desejada da tabela
contador = 0
for frase in l_test:
    
    probTDadoR = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_rel_absoluta:
            probTDadoR*=(tab_rel_absoluta[palavra] + 1) / (len(tab_relevante) + len(palavras_separadas_total))
        else:
            probTDadoR*=(0 + 1) / (len(tab_relevante ) + len(palavras_separadas_total))
                    
    probTDadoI = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_irr_absoluta:
            probTDadoI*=(tab_irr_absoluta[palavra] + 1) / (len(tab_irrelevante) + len(palavras_separadas_total))
        else:
            probTDadoI*=(0 + 1) / (len(tab_irrelevante) + len(palavras_separadas_total))
      
    probRdadoT = probTDadoR*probR  
    probIdadoT = probTDadoI*probI
 
  # Comparação das probabilidades e montando a tabela nova
    if probRdadoT>probIdadoT:
        
        NovoDF.loc[contador,"TweetN"]= " ".join(frase)
        NovoDF.loc[NovoDF.TweetN==" ".join(frase),"N_Classificação"] = "R"
    
    elif probRdadoT<probIdadoT:
        
        NovoDF.loc[contador,"TweetN"]= " ".join(frase)
        NovoDF.loc[NovoDF.TweetN==" ".join(frase),"N_Classificação"] = "I"
    
    contador+=1

###  Verificando a performance do classificador e resultados:

Agora você deve testar o seu classificador com a base de Testes.

In [23]:
# Criação de nova coluna 
NovoDF["Performance"] = " "

# verificação e preenchendo da tabela nova
for i in range(len(NovoDF)):
    if NovoDF.iloc[i, 1] == test.iloc[i,1]:
        if NovoDF.iloc[i, 1] == "R": NovoDF.iloc[i, 2]="Vdd_Positivo"
            
        else: NovoDF.iloc[i, 2]="Vdd_Negativo"  
    
    elif NovoDF.iloc[i, 1] != test.iloc[i,1]:       
        if NovoDF.iloc[i, 1] == "R": NovoDF.iloc[i, 2]="Falso_Positivo"
            
        else: NovoDF.iloc[i, 2]="Falso_Negativo"

In [24]:
NovoDF

,TweetN,N_Classificação,Performance
0,unico jogo novo anunciado pela sony que eu que...,I,Falso_Negativo
1,sony no seu nivel mais belo ne kkkkkkkkkk,R,Falso_Positivo
2,a sony vai lancar uma sequencia de jogos mt fo...,R,Falso_Positivo
3,gloria a sony,I,Vdd_Negativo
4,sony liberou a versao do ps5 fonte google,I,Vdd_Negativo
...,...,...,...
195,pois e kkkk ao menos a sony podia ter especifi...,I,Falso_Negativo
196,funimation encomendando dublagens a todo vapor...,R,Falso_Positivo
197,quer saber tudo o que aconteceu na showcase da...,I,Vdd_Negativo
198,o playstation 5 bateu o recorde do playstation...,I,Vdd_Negativo


In [25]:
# Calculo das porcentagens
p = NovoDF.Performance.value_counts(normalize=True).round(4)*100
vp = p["Vdd_Positivo"].round(4)
fp = p["Falso_Positivo"]
vn = p["Vdd_Negativo"]
fn = p["Falso_Negativo"]
print("{0}% Verdadeiro Positivo".format(vp))
print("{0}% Verdadeiro Negativo".format(vn))
print("{0}% Falso Negativo".format(fn))
print("{0}% Falso Positivo".format(fp))

14.0% Verdadeiro Positivo
47.5% Verdadeiro Negativo
17.5% Falso Negativo
21.0% Falso Positivo


In [27]:
NovoDF['N_Classificação'].value_counts()

I    130
R     70
Name: N_Classificação, dtype: int64

___
### Concluindo

In [28]:
#Distrubuição de acertos
d_Acertos= pd.crosstab(NovoDF["N_Classificação"],NovoDF['Performance'], normalize=True).round(4)*100
d_Acertos

Performance,Falso_Negativo,Falso_Positivo,Vdd_Negativo,Vdd_Positivo
N_Classificação,,,,
I,17.5,0.0,47.5,0.0
R,0.0,21.0,0.0,14.0


In [31]:
# Acertos e erros
acerto = p["Vdd_Positivo"] + p["Vdd_Negativo"]
acertos = acerto.round(5)
erro = p["Falso_Positivo"] + p["Falso_Negativo"]
erros = erro.round(5)
print("A porcentagem de acerto é de {0}%".format(acerto))
print("A porcentagem de erro é de {0}%".format(erro))

A porcentagem de acerto é de 61.5%
A porcentagem de erro é de 38.5%


### Análise:

   Para o desenvolvimento do projeto, foi utilizado uma base de dados de 500 tweets, sendo eles 300 para base de treinamento e 200 para a base de teste. A base de teste tinha 130 tweets irrelevantes e 70 relevantes, O classificador obteve uma performance aceitável, visto que teve uma precisão de 61.5% de acerto, considerando pequeno o tamanho da base de dados utilizada e que não foi realizada a classificação avançada.
  
### Mensagens com dupla negação e sarcasmo:

   A dupla negação e o sarcasmo não são identificados, já que o classificador trata eles indiferentemente, não conseguindo interpretar o contexto das frases, assim fazendo com que o programa não consiga diferenciar a negação e o sarcasmo. O classificador interpreta a palavra como um elemento independente e não inserido em um contexto de frase ou texto. O único momento do projeto que foi percebido a influência do sarcasmo e dupla negação foi na classificação dos tweets no excel, feito manualmente pela dupla, onde foi realizado o próprio julgamento dos autores do projeto sobre o que deve ser relevante ou não.
    
### Conclusão:

   
    
#### Por qual motivo devem continuar a investir no nosso projeto:

   Tendo em vista o tamanho da base de dados, é possível expandi-la, tendo uma base de dados maior com mais tweets, onde será possível extrair mais informações relevantes, assim igualando a quantidades de tweets relevantes e irrelevantes, para que o classificador tenha um parâmetro melhor para definir um tweet como relevante ou irrelevante. O projeto tem como intuito ajudar a empresa "Sony" em varias áreas, como: marketing, desenvolvimento de filmes e desenvolvimento de jogos. Isso porque os tweets coletados têm mensagens negativas em relação aos jogos que poderiam ser mais elaborados, filmes que poderiam ter mais tempo de duração, entre outras informações extraídas. Por conta dessa disponibilidade de informações, o projeto pode acrescentar melhorias à empresa em vários aspectos, tendo assim respostas mais positivas em seus produtos, gerando mais vendas e maior lucro.

    
#### Diferente cenário para Naïve Bayes fora do contexto do projeto:

   * Detecção de fraudes: identificar se uma transação financeira é “legal” ou “suspeita”.
   * Programas de filtragem de spam: detectar se e-mail é “normal” ou “spam”.
   * Aprovação de crédito: classificar um cliente como de “alto”, “médio” ou “baixo” risco para a concessão de crédito.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://www.youtube.com/watch?v=F_gYj6fcSl4 **Ensina como remover os acentos**


https://gist.github.com/luizomf/54b58615cd674db44153470c369a8284 **git do Youtube**

***Contextualização do Naive Bayes:*** https://www.datageeks.com.br/naive-bayes/ & https://www.vooo.pro/insights/6-passos-faceis-para-aprender-o-algoritmo-naive-bayes-com-o-codigo-em-python/